## Dataset MNIST

In [ ]:
from sklearn.datasets import fetch_openml
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# mnist = fetch_openml('mnist_784', version=1)
# mnist.keys()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/Inteligencia Artificial/IA - Clases de Práctica/PRACTICA_2023/Semana11-Clustering/Notebooks/'

In [ ]:
# mnist.target.unique()

In [ ]:
# mnist_784 = {}
# mnist_784["data"] = mnist.data.to_numpy()
# mnist_784["target"] = mnist.target.to_numpy()
# sio.savemat('../data/mnist_784.mat', mnist_784)

In [ ]:
a = sio.loadmat('../data/mnist_784.mat')
X, y = a["data"], a["target"].flatten()
#y = np.array([s[0].astype(int) for s in y])
print(X.shape)
print(y.shape)

In [ ]:
# tomo un dígito (fila de la matriz X) y lo redimensiono para llevarlo a una matriz de 28x28
some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.show()

In [ ]:
y[0]

**Este dataset ya está mezclado y dividido en entrenamiento y prueba**

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

## 1er Enfoque: Entrenamiento de Clasificador Binario

### Detector de 5

In [ ]:
y_train_5 = np.where(y_train == '5', True, False) 
y_test_5 = np.where(y_test == '5', True, False)

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000,  random_state=42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
sgd_clf.predict(X_train[0:1])

In [ ]:
X_train[0:1].shape #vector fila

### Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score

p = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy') #retorna los scores de evaluación 0.95035, 0.96035, 0.9604
print(p)
print(np.mean(p))

### **comparo resultado de cross_val_score y cross_val_predict**

`cross_val_predict`, retorna las predicciones realizadas en cada test fold, se obtiene una predicción 'limpia'. 

se dice limpia porque para cada instancia en el conjunto de entrenamiento la predicción se realiza por un clasificador que nunca vió esa muestra durante el entrenamiento.

In [ ]:
from sklearn.model_selection import cross_val_predict

predicciones = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [ ]:
n_correct = sum(predicciones == y_train_5)
print(n_correct / len(y_train_5))

**Este resultado por arriba del 95% pareciera decirnos que nuestro clasificador es muy bueno detectando el número 5. Probemos un clasificador dummy que solo clasifique cada imagen en la clase no-5**

In [ ]:
from sklearn.base import BaseEstimator

class DummyClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros(( len(X), 1 ), dtype=bool)

**La predicción sólo retorna un vector de Falses**

In [ ]:
dummy_clf = DummyClassifier()

cross_val_score(dummy_clf, X_train, y_train_5, cv=3, scoring='accuracy')

**Esto tiene sentido porque sólo 10% de las imágenes en el dataset son 5**

**Si nos preguntamos si una imagen del dataset no es 5, estaremos en lo correcto 90% de las veces**

**Accuracy no es una buena medida de performance del clasificador cuando trabajamos con un conjunto de datos no balanceado (algunas clases tienen más datos que otras)**

### Matriz de confusión

**para obtener la matriz de confusión necesitamos realizar predicciones para poder compararlas con los valores reales, dejamos sin tocar el conjunto de prueba y usamos validación cruzada en el conjunto de entrenamiento**

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(y_train_5, y_train_pred), columns=['pred.Neg(no 5s)', 'pred.Pos(5s)'],
             index= ['clase Neg(no 5s)', 'clase Pos(5s)'])

**cada fila en la matriz representa la clase verdadera, cada columna la clase que se predice**

**la primera fila representa la clase no-5, 53892 fueron correctamente clasificados como no-5s (verdaderos negativos), 687 fueron mal clasificados como 5s (Falsos positivos)**

**la segunda fila representa la clase 5, 1891 fueron clasificados incorrectamente como no-5s (Falsos negativos), los 3530 restantes fueron clasificados correctamente como 5s (Verdaderos positivos)**

### Precisión (Tasa de aciertos de las predicciones positivas)

$$precision = \frac{VP}{VP+FP}$$

### Recall (Sensibilidad, TPR)

$$recall = \frac{VP}{VP+FN}$$

Precisión: razón o proporción dentro de las predicciones positivas que son verdaderamente positivas

Sensibilidad: razón o proporción de las instancias positivas que son detectadas correctamente por el clasificador

<center><img src="https://drive.google.com/uc?export=view&id=1rYcBjJvK6sRi3XCBKIQS-80st0LbOHxR" width=900 alt="centered image"></center>


In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)

In [ ]:
cm = confusion_matrix(y_train_5, y_train_pred)
#precision
cm[1, 1] / (cm[0, 1] + cm[1, 1])

In [ ]:
#sensibilidad
recall_score(y_train_5, y_train_pred)

In [ ]:
cm[1, 1] / (cm[1, 0] + cm[1, 1])

**Entonces nuestro clasificador cuando dice que predijo un 5, está en lo cierto sólo un 84% de las veces. Además solo detecta el 65% de las instancias que son 5s.**

**Es más conveniente combinar ambas métricas en una sóla llamada F1 score, especialmente si necesitamos una forma simple de comparar dos clasificadores**

### F1 score

$$F1 = \frac{2}{\frac{1}{precision} + \frac{1}{recall}} = 2 \times \frac{precision \times recall}{precision + recall} = \frac{VP}{VP + \frac{FN + FP}{2}}$$

Es la media armónica de la precisión y el recall. A diferencia de la media común, es mucho más sensible a valores pequeños. El clasificador tendrá un alto valor F1 sólo si la precisión y el recall tienen ambos valores altos.

No siempre buscamos tener una precisión y recall similares. En algunos casos nos interesa más la precisión y en otros el recall. 

Por ejemplo, si entrenamos un clasificador para detectar videos aptos para niños, es preferible un clasificador que rechace muchos videos que son aptos (baja sensibilidad - muchos FN) pero acepte sólo los videos que son aptos (alta precisión - muchos VP).

Por otra parte si queremos entrenar un clasificador que detecte ladrones en imágenes de vigilancia, probablemente es aceptable una baja precisión mientras la sensibilidad sea alta (si hay un ladrón quiero que lo detecte)

Desafortunadamente no se puede tener ambos a la vez: aumentar la precisión reduce la sensibilidad.

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)


<center><img src="https://drive.google.com/uc?export=view&id=1rTStqcXeLux4I1jLQ8a739ELcr5ZzUSR" width=1000 alt="centered image"></center>



In [ ]:
#puedo acceder al valor que utiliza el clasificador para realizar la predicción 

y_scores = sgd_clf.decision_function([some_digit])
y_scores

In [ ]:
umbral = 0
y_pred_digito = (y_scores > umbral)
y_pred_digito

**El clasificador usa un umbral 0, así que el código anterior retorna el mismo resultado que el método predict()**

In [ ]:
umbral = 8000
y_pred_digito = (y_scores > umbral)
y_pred_digito

**cómo se decide el umbral a utilizar?**

puedo obtener los valores de las predicciones calculadas por el clasificador para todas las instancias de entrenamiento

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method = 'decision_function')
y_scores

**podemos graficar la curva de precisión - recall para todos los umbrales posibles**

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, umbrales = precision_recall_curve(y_train_5, y_scores)

In [ ]:
print(precisions.shape)
print(recalls.shape)
print(umbrales.shape)

In [ ]:
def plot_precision_recall_vs_umbrales(precisions, recalls, umbrales):
    plt.plot(umbrales, precisions[:-1], 'b--', label='Precision', linewidth=2)
    plt.plot(umbrales, recalls[:-1], 'g-', label='Recall', linewidth=2)
    plt.legend(loc='center right', fontsize=16)
    plt.xlabel('Umbral', fontsize=16)
    plt.grid(True)
    plt.axis([-50000, 50000, 0, 1])

In [ ]:
plt.figure(figsize=(13,7))
plot_precision_recall_vs_umbrales(precisions, recalls, umbrales)


**suponer queremos un 90% de precisión**

In [ ]:
umbral_90_precision = umbrales[np.argmax(precisions >= 0.9)]
umbral_90_precision

In [ ]:
y_90_pred = (y_scores >= umbral_90_precision)
y_90_pred

In [ ]:
precision_score(y_train_5, y_90_pred)

In [ ]:
recall_90_precision = recall_score(y_train_5, y_90_pred)
recall_90_precision

In [ ]:
#esto no es necesario
recall_90_precision = recalls[np.argmax(precisions >= 0.9)]
recall_90_precision

In [ ]:
plt.figure(figsize=(13,7))
plot_precision_recall_vs_umbrales(precisions, recalls, umbrales)
#Graficar lo que sigue después
plt.plot([umbral_90_precision, umbral_90_precision],[0, 0.9], 'r:')
plt.plot([-50000, umbral_90_precision],[0.9, 0.9], 'r:')
plt.plot([-50000, umbral_90_precision],[recall_90_precision, recall_90_precision], 'r:')
plt.plot([umbral_90_precision], [0.9], 'ro')
plt.plot([umbral_90_precision], [recall_90_precision], 'ro')
plt.title('precision_recall_vs_umbrales')
plt.show()

**Curva precisión vs recall (PR)**

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, 'b-', linewidth=2)
    plt.xlabel('Recall', fontsize=16)
    plt.ylabel('Precision', fontsize=16)
    plt.axis([0,1,0,1])
    plt.grid(True)

In [ ]:
plt.figure(figsize=(13,7))
plot_precision_vs_recall(precisions, recalls)

plt.plot([recall_90_precision, recall_90_precision],[0, 0.9], 'r:')
plt.plot([0, recall_90_precision],[0.9, 0.9], 'r:')
plt.plot([recall_90_precision], [0.9], 'ro')
plt.title('precision_vs_recall')
plt.show()

### La curva ROC (Receiver operating characteristic)

**para clasificadores binarios**

**Se grafica la tasa de VP (recall) en función de la tasa de falsos positivos (TFP). La TFP es la proporción de instancias negativas que son incorrectamente clasificadas como positivas (= 1 - TVN )**

**TVN también se conoce como especificidad**

**La curva ROC grafica _sensibilidad_(recall) vs 1-_especificidad_**

La curva ROC representa la tasa de verdaderos positivos o Recall en función de la tasa de falsos positivos esto además para distintos umbrales del clasificador. El umbral es un valor que utiliza el clasificador para decidir si una predicción es positiva o no. Dependiendo del clasificador, se utilizan las funciones predict_proba o decision_function.

In [ ]:
from sklearn.metrics import roc_curve

tfp , tvp , umbrales = roc_curve(y_train_5, y_scores)

In [ ]:
def plot_curva_roc(tfp, tvp, label=None):
    plt.plot(tfp, tvp, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
    plt.axis([0, 1, 0, 1])                                    
    plt.xlabel('Tasa de falsos positivos', fontsize=16) 
    plt.ylabel('Tasa de verdaderos positivos (Recall)', fontsize=16)    
    plt.grid(True)         

In [ ]:
plt.figure(figsize=(13, 8))                                    
plot_curva_roc(tfp, tvp)

tfp_90 = tfp[np.argmax(tvp >= recall_90_precision)]    
plt.plot([tfp_90, tfp_90], [0., recall_90_precision], "r:")
plt.plot([0.0, tfp_90], [recall_90_precision, recall_90_precision], "r:")  
plt.plot([tfp_90], [recall_90_precision], "ro")               
plt.title('curva_roc')                                  
plt.show()

**La línea de puntos representa la curva roc de un clasificador que predice por azar, un buen clasificador se mantiene lo más alejado posible de esta curva (hacia la esquina superior izquierda)**

**Una forma de comparar clasificadores es medir el área bajo la curva (AUC) roc, un clasificador perfecto tendrá un área de 1, un clasificador por azar 0.5**

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_5, y_scores)

**Cual de estas 2 curvas utilizar?**

**Utilizar la curva PR cuando la clase positiva sea escasa o cuando me interese más los falsos positivos que los falsos negativos (ejemplo de videos)**

**En caso contrario usar la curva ROC**

**Por ejemplo con la curva ROC podríamos creer que nuestro clasificador es bueno (0.96 AUC) pero esto se debe principalmente a que se tienen pocas instancias verdaderas (5s). Si usamos la curva PR nos queda claro que el clasificador tiene opción a mejorar**